In [5]:
##videos
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import operator
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline
folder = "test_images/"
images = os.listdir(folder)




#Converts to Grayscale. returns a grayscale image
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def color_dropout(image): 
    # Grab the x and y size and make a copy of the image
    ysize = image.shape[0]
    xsize = image.shape[1]
    color_select = np.copy(image)

    # Define color selection criteria
    ###### MODIFY THESE VARIABLES TO MAKE YOUR COLOR SELECTION
    red_threshold = 150
    green_threshold = 150
    blue_threshold = 150
    ######

    rgb_threshold = [red_threshold, green_threshold, blue_threshold]

    # Do a boolean or with the "|" character to identify
    # pixels below the thresholds
    thresholds = (image[:,:,0] < rgb_threshold[0]) \
                | (image[:,:,1] < rgb_threshold[1]) \
                | (image[:,:,2] < rgb_threshold[2])
    color_select[thresholds] = [0,0,0]    
    return color_select
    

#define the mask for the image. Returns a masked image
def region_of_interest(img, vertices):

    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

##perform the Gaussian conversion. Returns an image
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

##performs canny edge detection. Returns an image
def cannyEdges(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)



##Uses Hough space to determine lines and draw them on the image.  Returns an image with 1 right and 1 left line
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)   
    draw_lines(line_img, lines)
    #draw_hough_lines(line_img, lines)
    return line_img


#used for testing to see where the lines are found at. 
def draw_hough_lines(img, lines, color=[255, 0, 0], thickness=2):

    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
#Draws the actual lines on the image
#returns an image with one line for each the left and right sides.
def draw_lines(img, lines):
    #determine the slope of each line.
    linesAndSlopeList = determine_slope(lines) 
    topOfMask = img.shape[0] * .60
    #Noe we need to split the lines into the left and right slope
    leftLinesList = []
    rightLinesList = []
    #now we have an array with coordinates and slope
    if linesAndSlopeList is not None:
        for i in linesAndSlopeList:
            try:             
                if i[4]>=0:
                    rightLinesList.append(i)
                else:
                    leftLinesList.append(i)
            except IndexError:
                continue
    
    newMaskColor = [51, 51, 255]  
    thickness=10 
    imshape = img.shape
    """
    Here is where we will see if there are any lines at all. 
    if we dont have anything in the list len(list) == 0, then use the previous lines.
    
    """ 
    
    
    if (len(leftLinesList) == 0 ):
        cv2.line(img, (int(previousLeftLineList[0][0]), int(previousLeftLineList[0][1])), (int(previousLeftLineList[0][2]), int(previousLeftLineList[0][3])), newMaskColor, thickness)
    elif len(leftLinesList) > 0 :
        #print(leftLinesList)
        leftLinesAvgList = determine_lines_within_average_slope(leftLinesList)
        #print(leftLinesAvgList)
        averageX = (np.average(leftLinesAvgList, axis=0)[0]+np.average(leftLinesAvgList, axis=0)[2])/2
        averageY = (np.average(leftLinesAvgList, axis=0)[1]+np.average(leftLinesAvgList, axis=0)[3])/2 
        averageSlopeLeft = np.average(leftLinesAvgList, axis=0)[4]
        newLineLeft = extrapolate_lines(averageX, averageY, averageSlopeLeft, imshape[0], topOfMask + 25)
        previousLeftLineList.clear()
        previousLeftLineList.append(newLineLeft)
        cv2.line(img, (int(newLineLeft[0]), int(newLineLeft[1])), (int(newLineLeft[2]), int(newLineLeft[3])), newMaskColor, thickness)
    
    if (len(rightLinesList) == 0 & len(previousRightLineList) > 0):
        cv2.line(img, (int(previousRightLineList[0][0]), int(previousRightLineList[0][1])), (int(previousRightLineList[0][2]), int(previousRightLineList[0][3])), newMaskColor, thickness)
        
    elif len(rightLinesList) > 0 :
        rightLinesAvgList = determine_lines_within_average_slope(rightLinesList)
        #print("AverageList:", rightLinesAvgList)
        averageX = (np.average(rightLinesAvgList, axis=0)[0]+np.average(rightLinesAvgList, axis=0)[2])/2
        averageY = (np.average(rightLinesAvgList, axis=0)[1]+np.average(rightLinesAvgList, axis=0)[3])/2 
        averageSlopeRight = np.average(rightLinesAvgList, axis=0)[4]
        newLineRight = extrapolate_lines(averageX, averageY, averageSlopeRight, imshape[0], topOfMask + 25)
        previousRightLineList.clear()
        previousRightLineList.append(newLineRight)
        cv2.line(img, (int(newLineRight[0]), int(newLineRight[1])), (int(newLineRight[2]), int(newLineRight[3])), newMaskColor, thickness)


        
#lines from the hough transformation do not have the slope attached to them
#returns an array of lines with slopes.
def determine_slope(lines):
    #get the slope for each line
    #print(lines)
    allLines = []
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                slope = (y2-y1)/(x2-x1)
                slopeLine = [x1,y1,x2,y2,slope]
                allLines.append(slopeLine)
        return allLines  



#Finds the average slope of all the left or right lines
# it then removes any which are above or below the threshold
def determine_lines_within_average_slope(lines):    
    returnLines = []
    if len(lines) >= 1:
        returnLines.clear()
        linesToRemove = []
        for i in lines:
            if i[4] == 0 or i[0] == i[2] or i[1] == i[3]:
                linesToRemove.append(i)
           
        for i in linesToRemove:
            lines.remove(i)
        #print("after removal", lines)
        averageSlope = np.mean(lines, axis = 0)[4]
        #print("avg slope", averageSlope)
        #print(averageSlope)
        lowThreshold = averageSlope - .3
        highThreshold = averageSlope + .3        
        #print("low", lowThreshold)
        #print("high", highThreshold)
        for i in lines:
            if i[4]>=lowThreshold and i[4] <= highThreshold:
                returnLines.append(i)
        return returnLines
    else:
        return returnLines
#takes a partial line and extrapolates it out to the top and bottom of the ROI mask
def extrapolate_lines(x, y, slope, bottomY, topY):
    yIntercept = y - (slope * x)
    topX = (topY - yIntercept)/slope
    bottomX = (bottomY - yIntercept)/slope   
    newLine = [int(topX), int(topY), int(bottomX), int(bottomY)]
    return newLine


def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)
   
def process_image(image):

    return read_and_mark_image(image)

def read_and_mark_image(image):
    
    originalImage = image
    lowThreshold = 50
    highThreshold = 100
    topOfMask = 300
    rho =5
    theta = np.pi/180
    threshold = 200
    min_line_length = 100
    max_line_gap = 1
    
    #1. Convert to grayscale
    gray = grayscale(image)
             
            
    #2: do a color dropout.
   
    
    #dropImage = color_dropout(image)
    #return dropImage
    
    blurImage = gaussian_blur(image, 3)
    
    #3: perform canny edge detection
    lowThreshold = 50
    highThreshold = 150
    canny = cannyEdges(blurImage, lowThreshold, highThreshold)
    
    imshape = canny.shape
    
    topOfMask = image.shape[0] * .6
    maskTopLeftX = (image.shape[1] / 2) - (image.shape[1] * .05)
    maskTopRightX = (image.shape[1] / 2) + (image.shape[1] * .05)
    MaskBottomLeftX = image.shape[1] * .15
    MaskBottomRightX = image.shape[1] * .95    
    #4: apply the image mask 
    #vertices = np.array([[(115,imshape[0]),(470, topOfMask), (500,topOfMask), (900,imshape[0])]], dtype=np.int32)
    vertices = np.array([[(MaskBottomLeftX, imshape[0]),(maskTopLeftX, topOfMask), (maskTopRightX, topOfMask), (MaskBottomRightX, imshape[0])]], dtype=np.int32)
    
    
    maskedImage = region_of_interest(canny, vertices)
    #return maskedImage
    #5 Perform the Hough Transfer, 
    rho = 5
    theta = np.pi/180
    threshold = 100
    min_line_length = 10
    max_line_gap = 1
    houghImage = hough_lines(maskedImage, rho, theta, threshold, min_line_length, max_line_gap )   
    
    finalImage =  weighted_img(houghImage, originalImage)
    
    return finalImage


"""
setting some variables here to deal with different resolution images... Prob dont have to do this in real life but I like one
code base to run for everything.
"""


previousLeftLineList = []
previousLeftLineList.append([0,0,0,0])
previousRightLineList = []
previousRightLineList.append([0,0,0,0])
allLines = []


imagesTest = False

if imagesTest == True:    
    for imageFile in images:            
        image = mpimg.imread(folder + imageFile)
        displayImage = read_and_mark_image(image)
        plt.figure()
        plt.imshow(displayImage, cmap='Greys_r')
        plt.imshow(displayImage)
        #filename, ext = os.path.splitext(imageFile)
        #new_filename = os.path.join(folder, filename + "_out" + ext)
        #mpimg.imsave(new_filename, displayImage, cmap='Greys_r')


else:
        
    output = 'solidYellowLeft_OUT.mp4'
    clip1 = VideoFileClip("solidYellowLeft.mp4")
    clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    %time clip.write_videofile(output, audio=False)
    HTML("""
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(output))


[MoviePy] >>>> Building video solidYellowLeft_OUT.mp4
[MoviePy] Writing video solidYellowLeft_OUT.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:24<00:00, 27.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: solidYellowLeft_OUT.mp4 

Wall time: 25.8 s


Reflection:
This was my first time using Python, and it was quite a challenge.  I do a lot of .NET development, and I had to give up a lot of the tools I use, such as being able to step through the code and see how variables are actually being manipulated.  Python was also very confusing as variables are never directly instantiated as a type.  Meaning I could take a variable named "Joe" and make it a string, then an int, and then finally a list.  There was a lot of time spent printing out variables to see what I was actually working with.

Some of the challenges I dealt with were just dealing with the basic Python syntax.  Most of the code I had planned out had to be rewritten due to various errors and access issues (making class variables). I also had to deal with images of varying sizes, which caused me to rewrite my ROI mask to use percentages instead of fixed pixels.  This allowed it to scale properly no matter what image size i used.  I can image this would not be an issue in a real world application since the picture will be a fixed width.

Now on to the project itself. I made my pipeline (read_and_mark_images()) and went through the basic steps I was taught in class.  Perform the gray scale conversion, gaussian smoothing, canny edge detection, and finally a weighted image returned to the user.

In order to find the lane lines, I did a few things.  First I split all the lines according to if the slope was + or 1, which gave me the left and right lines.  I then took the lines and determined an average slope for them, and removed all outliers which werent within a specific variance (.3).  Once I had the outliers removed, i had to determine if there were any lines actually in the list.  If there were, then I took the average coordinates and slope of the lines, and used a function extrapolate_lines() to create a full line. This method took the average coordinates and slope and used them to find the X coordinates for the top of the mask and the bottom of the image.  These were all we needed to find, since the Y coordinates came from the mask and the image itself.  A few times I ran into an issue where there were no lines identified.  For these circumstances I used the lines I previously found and drew them on the image until a new set of coordinates were found.

For the extrapolate lines, I decided to use the averages and only draw one line for the left and one for the right.  I figured this would be much faster when it came to rendering the video. 

The pipeline seemed pretty smooth and there wasnt an extremely long time to render the video. however when trying it on the extra challenge video, there were a few areas where I missed the lines.  I also noticed the slopes I found were around .67 (- or +) so I am curious if this would be a good median threshold to use when determining if the lines i found are the lane lines or another random line. 

Overall, I found this to be a rather challenging project and I was very excited to complete it!  